In [1]:
import pandas as pd
import numpy as np

import Clean_data as cld
import Transform_Data as tfd
import Visualize_Data as vld

In [2]:
st_degree_ct_df =pd.read_csv('Final_Data/Melted_State_Degree_Ratios.csv')
indst_grp = pd.read_csv('Final_Data/Industry_counts_global.csv')

Step one is to remove the raw counts, aside from over-all bachelors degree holders. this will reduce the weight of pitting larger and smaller states against each other.

In [3]:
state_totals = st_degree_ct_df[st_degree_ct_df['field'] == 'State_Total']

In [4]:

state_totals['bachelor_weight'] = np.log(state_totals[ 'ratio'])

C:\Users\joshf\AppData\Local\Temp/ipykernel_7012/801825247.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_totals['bachelor_weight'] = np.log(state_totals[ 'ratio'])


In [5]:
def min_max_scaling(column):
    return (column - column.min())/ (column.max() - column.min())

In [6]:
state_totals['normalize_bachelor_weight'] = min_max_scaling(state_totals['bachelor_weight'])

C:\Users\joshf\AppData\Local\Temp/ipykernel_7012/2506825057.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_totals['normalize_bachelor_weight'] = min_max_scaling(state_totals['bachelor_weight'])


In [7]:
state_totals.to_csv('Final_Data/Weighted_bachelor_total_state.csv')


### Part 2

In [8]:
state_ratios = st_degree_ct_df[st_degree_ct_df['field'] != 'State_Total']

In [9]:
state_ratios.head(4)

,Unnamed: 0,state,field,ratio
51,51,District of Columbia,"Science, Technology, Engineering",0.485053
52,52,Washington,"Science, Technology, Engineering",0.421292
53,53,California,"Science, Technology, Engineering",0.416775
54,54,Maryland,"Science, Technology, Engineering",0.408777


In [10]:
state_ratios.to_csv('Final_Data/Weighted_Degree_Ratios_State.csv')

### Part 3

In [11]:
ratio_grp = state_ratios.groupby('field')['ratio'].mean()

In [12]:
ratio_grp

field
Architecture, Design                    0.103498
Business, Admin, PR                     0.193936
Consumer, Retail, Arts, Humanities      0.224471
Health, Education, Government, Civil    0.136296
Science, Technology, Engineering        0.341798
Name: ratio, dtype: float64

In [13]:
indst_grp.head(4)

,Unnamed: 0,industry,count,field
0,0,information technology and services,534948,"Science, Technology, Engineering"
1,1,marketing and advertising,385649,"Business, Admin, PR"
2,2,real estate,274165,"Business, Admin, PR"
3,3,management consulting,242239,"Business, Admin, PR"


In [14]:
indst_grp = indst_grp.groupby('field')['count'].sum()

In [15]:
indst_grp = indst_grp.to_frame().reset_index()

In [16]:
indst_grp

,field,count
0,"Architecture, Design",848572
1,"Business, Admin, PR",1995790
2,"Consumer, Retail, Arts, Humanities",2363082
3,"Health, Education, Government, Civil",1395824
4,"Science, Technology, Engineering",2358322


In [17]:
indst_grp['normalize_industry_weight'] = np.log(indst_grp['count'])

In [18]:
indst_grp

,field,count,normalize_industry_weight
0,"Architecture, Design",848572,13.651310
1,"Business, Admin, PR",1995790,14.506551
2,"Consumer, Retail, Arts, Humanities",2363082,14.675477
3,"Health, Education, Government, Civil",1395824,14.148995
4,"Science, Technology, Engineering",2358322,14.673461


In [19]:
indst_grp.to_csv('Final_Data/Weighted_Industry_Fields.csv')

### Part 4


From the previous notebook, I can now bring in some data to help apply weights

In [20]:
degree_specs_placement= pd.read_csv('Datasets/Discoveries/Degree_Specs_Pruned_Placement.csv')
degree_payback = pd.read_csv('Datasets/Discoveries/Degree_payback_grouped_mean.csv')

In [21]:
degree_specs_placement.head(4)

,early_specialisation,degree,specialisation,mean,count,median,std,min,max
0,Science,Sci&Tech,Mkt&Fin,322720.0000,25,275000,115811.45600,200000,690000
1,Science,Sci&Tech,Mkt&HR,304066.6667,15,276000,72335.00108,210000,450000
2,Commerce,Comm&Mgmt,Mkt&Fin,299685.1852,54,277500,116492.74400,200000,940000
3,Commerce,Comm&Mgmt,Mkt&HR,260541.6667,24,260000,39139.63761,200000,380000


Again, discard this dataset because it's too limited in scope

In [22]:
degree_payback.sort_values(by='Starting Median Salary',ascending=False)

,Undergraduate Major,Starting Median Salary,Mid-Career Median Salary,Percent change from Starting to Mid-Career Salary,field
43,Physician Assistant,74300,91700,0.234,"Science, Technology, Engineering"
8,Chemical Engineering,63200,107000,0.693,"Science, Technology, Engineering"
12,Computer Engineering,61400,105000,0.710,"Science, Technology, Engineering"
19,Electrical Engineering,60900,103000,0.691,"Science, Technology, Engineering"
38,Mechanical Engineering,57900,93600,0.617,"Science, Technology, Engineering"
1,Aerospace Engineering,57700,101000,0.750,"Science, Technology, Engineering"
30,Industrial Engineering,57700,94700,0.641,"Science, Technology, Engineering"
13,Computer Science,55900,95500,0.708,"Science, Technology, Engineering"
40,Nursing,54200,67000,0.236,"Health, Education, Government, Civil"
10,Civil Engineering,53900,90500,0.679,"Science, Technology, Engineering"


In [23]:
degree_payback.to_csv('Final_Data/Baseline_Degree_payback_grouped_mean.csv')

These numbers come in much lower than other databases I've used. I'm going to use these as baseline numbers.